In [7]:
import pandas as pd
import json
from openai import OpenAI
from tqdm import tqdm
import time
import os

In [ ]:
def translate_item(client, item, index, pbar=None):
    """단일 항목을 번역하는 함수"""
    try:
        # 번역 요청 구성
        system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
        다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
        user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
        출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.
[입력]
{json.dumps(item, ensure_ascii=False)}
번역 결과만 제공해주세요."""

        # 로그 출력
        print(f"\n[번역 시작 - 항목 {index}]")
        start_time = time.time()

        # API 호출로 번역
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
            response_format={"type": "json_object"},
        )

        # 번역 결과 적용
        translated_text = response.choices[0].message.content.strip()
        translated_item = json.loads(translated_text)

        # 소요시간 및 완료 로그
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[번역 완료 - 항목 {index}] ({elapsed_time:.2f}초 소요)")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"처리 중: {index}")

        return translated_item

    except Exception as e:
        error_msg = f"Error occurred at index {index}: {e}"
        print(f"\n[에러 - 항목 {index}] {error_msg}")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"에러 발생: {index}")

        return None


def translate_english_to_korean(data_list):
    """
    List[dict] 형태의 데이터에서 영어로 된 'user_input'과 'reference' 필드를 한국어로 번역합니다.
    각 항목을 개별적으로 API 호출하여 번역합니다.
    Args:
        data_list: 번역할 List[dict] 형태의 데이터
    Returns:
        번역된 데이터가 포함된 List[dict]
    """
    # OpenAI 클라이언트 초기화
    client = OpenAI()

    # 결과 저장 리스트
    translated_data = []

    # 총 항목 수와 처리 현황 계산 변수
    total_items = len(data_list)
    success_count = 0
    error_count = 0

    print(f"\n[번역 시작] 총 {total_items}개 항목 처리 예정")

    # tqdm으로 진행상황 표시
    with tqdm(total=total_items, desc="번역 진행률") as pbar:
        for index, item in enumerate(data_list):
            # 항목 번역
            translated_item = translate_item(client, item, index, pbar)

            # 결과 처리
            if translated_item:
                translated_data.append(translated_item)
                success_count += 1
            else:
                error_count += 1

            print(translated_item)

            # 10개 항목마다 중간 상황 보고
            if (index + 1) % 10 == 0 or index == total_items - 1:
                print(
                    f"\n[진행 상황] {index + 1}/{total_items} 완료 (성공: {success_count}, 실패: {error_count})"
                )

    # 최종 결과 요약
    print(
        f"\n[번역 완료] 총 {total_items}개 중 {success_count}개 성공, {error_count}개 실패"
    )

    return translated_data


def main():
    # 예시 데이터 - List[dict] 형태
    file_paths = [
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv",
    ]

    print("[데이터 로드 시작]")
    df_list = []
    for file in file_paths:
        print(f"파일 로드 중: {file}")
        df = pd.read_csv(file)
        df_list.append(df)
        print(f"파일 로드 완료: {file} ({len(df)}개 행)")

    concatenated_data = pd.concat(df_list)
    print(
        f"[데이터 로드 완료] 총 {len(concatenated_data)}개 행, {concatenated_data.shape[1]}개 열"
    )

    # 데이터프레임을 딕셔너리 리스트로 변환
    data_list = concatenated_data.to_dict("records")

    # 번역 실행
    print("[번역 프로세스 시작]")
    translated_data = translate_english_to_korean(data_list)

    # JSON 형식으로 변환하여 출력 및 저장
    translated_json = json.dumps(translated_data, ensure_ascii=False, indent=2)

    print("\n[결과 저장 중]")
    output_file = "translated_output.json"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(translated_json)

    print(
        f"\n[작업 완료] {len(translated_data)}개의 데이터가 번역되어 '{output_file}' 파일로 저장되었습니다."
    )


if __name__ == "__main__":
    main()

In [8]:
def translate_item(client, item, index, pbar=None):
    """단일 항목을 번역하는 함수"""
    try:
        # 번역 요청 구성
        system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
        다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
        user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
        출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.
[입력]
{json.dumps(item, ensure_ascii=False)}
번역 결과만 제공해주세요."""

        # 로그 출력
        print(f"\n[번역 시작 - 항목 {index}]")
        start_time = time.time()

        # API 호출로 번역
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
            response_format={"type": "json_object"},
        )

        # 번역 결과 적용
        translated_text = response.choices[0].message.content.strip()
        translated_item = json.loads(translated_text)

        # 소요시간 및 완료 로그
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[번역 완료 - 항목 {index}] ({elapsed_time:.2f}초 소요)")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"처리 중: {index}")

        return translated_item

    except Exception as e:
        error_msg = f"Error occurred at index {index}: {e}"
        print(f"\n[에러 - 항목 {index}] {error_msg}")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"에러 발생: {index}")

        return None


def translate_english_to_korean(data_list, output_path="translated_output.jsonl"):
    """
    List[dict] 형태의 데이터에서 영어로 된 'user_input'과 'reference' 필드를 한국어로 번역합니다.
    각 항목을 개별적으로 API 호출하여 번역합니다.
    번역 결과는 매번 output_path에 append 저장합니다.
    이미 번역된 인덱스는 건너뜁니다.
    """
    from openai import OpenAI

    client = OpenAI()

    # 이미 번역된 데이터 읽기
    translated_indices = set()
    translated_data = []
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    item = json.loads(line)
                    translated_data.append(item)
                    if "index" in item:
                        translated_indices.add(item["index"])
                except Exception:
                    continue

    for index, item in enumerate(data_list):
        if index in translated_indices:
            continue  # 이미 번역된 항목은 건너뜀

        try:
            system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
            다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
            user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.

[입력]
{json.dumps(item, ensure_ascii=False)}

번역 결과만 제공해주세요."""

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=0.1,
                response_format={"type": "json_object"},
            )
            translated_text = response.choices[0].message.content.strip()
            translated_item = json.loads(translated_text)
            translated_item["index"] = index  # 인덱스 정보 추가

            # 결과를 파일에 append
            with open(output_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(translated_item, ensure_ascii=False) + "\n")

            translated_data.append(translated_item)
            print(f"✅ 번역 완료: {index + 1}/{len(data_list)}")
        except Exception as e:
            print(f"❌ Error at index {index}: {e}")
            continue

    return translated_data


def main():
    file_paths = [
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv",
    ]
    df_list = []
    for file in file_paths:
        df = pd.read_csv(file)
        df_list.append(df)
    concatenated_data = pd.concat(df_list)
    print(concatenated_data.shape)

    data_list = concatenated_data.to_dict("records")
    translated_data = translate_english_to_korean(
        data_list, output_path="translated_output.jsonl"
    )

    print(
        f"\n전체 번역 완료! {len(translated_data)}개 데이터가 번역되었습니다. 'translated_output.jsonl' 파일을 확인하세요."
    )


In [ ]:
if __name__ == "__main__":
    main()

In [18]:
result = []
# 'r'은 읽기 모드, encoding='utf-8'은 한글 등 깨짐 방지를 위해 명시해주는 것이 좋습니다.
with open("translated_output.jsonl", "r", encoding="utf-8") as f:
    # 파일을 라인 단위로 순회합니다. 파일 끝에 도달하면 자동으로 멈춥니다.
    for line in f:
        # 읽어온 라인 좌우의 공백(줄바꿈 문자 포함)을 제거합니다.
        stripped_line = line.strip()
        # 빈 라인이 아닌 경우에만 처리합니다.
        if stripped_line:
            try:
                # JSON 형식으로 파싱하여 리스트에 추가합니다.
                result.append(json.loads(stripped_line))
            except json.JSONDecodeError as e:
                # JSON 파싱 오류가 발생했을 경우, 어떤 라인에서 오류가 났는지 출력하고 건너뜁니다.
                print(f"Error decoding JSON on line: {stripped_line}. Error: {e}")
                continue  # 오류가 난 라인은 건너뛰고 다음 라인으로 이동

# result 리스트에 데이터가 있는 경우에만 데이터프레임을 생성하고 CSV로 저장합니다.
if result:
    # 리스트를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(result)
    print(df.columns)
    df = df.drop(columns="index")

    # 데이터프레임을 CSV 파일로 저장합니다.
    # index=False는 데이터프레임의 인덱스를 CSV에 쓰지 않도록 합니다.
    # encoding='utf-8'은 CSV 파일 저장 시 인코딩을 지정합니다.
    df.to_csv("translated_output.csv", index=False, encoding="utf-8")

    print("Successfully converted jsonl to csv.")
else:
    print("No valid data found in the jsonl file.")

Index(['user_input', 'reference_outputs', 'reference', 'synthesizer_name',
       'index'],
      dtype='object')
Successfully converted jsonl to csv.


(558, 4)


(558, 4)
